In [ ]:
# import libraries
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
from google.colab.patches import cv2_imshow
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from google.colab import files
from google.colab import drive


# load serialized face detector
print("Loading Face Detector...")
protoPath = "/content/drive/MyDrive/Myphotos/model/deploy.prototxt"
modelPath = "/content/drive/MyDrive/Myphotos/model/res10_300x300_ssd_iter_140000_fp16 (1).caffemodel"
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# load serialized face embedding model
print("Loading Face Recognizer...")
embedder = cv2.dnn.readNetFromTorch("/content/drive/MyDrive/Myphotos/model/openface_nn4.small2.v1.t7")

Loading Face Detector...
Loading Face Recognizer...
Quantifying Faces...


In [ ]:
# grab the paths to the input images in our dataset
print("Quantifying Faces...")
imagePaths = list(paths.list_images("/content/drive/MyDrive/Myphotos/FaceDataset"))

Quantifying Faces...


In [ ]:
knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

In [ ]:
TRAINING_SIZE=50
ThreshTime = 15

In [ ]:
def maxFreq(resp):
	MaxName={}
	for each in resp:
		if each in MaxName:
			MaxName[each]= MaxName[each]+1
		else:
			MaxName[each]= 0
	Max = 0
	ind =str()
	
	for key, val in MaxName.items():
		if(Max<val):
			Max = val
			ind = key
		else:
			pass
	return ind

In [ ]:
def embeddingsGenrerator():
  total=0
  for (i, imagePath) in enumerate(imagePaths[:5]):
    # extract the person name from the image path
    if (i%50 == 0):
      print("Processing image {}/{}".format(i, len(imagePaths)))
    name = imagePath.split(os.path.sep)[-2]

    # load the image, resize it to have a width of 600 pixels (while maintaining the aspect ratio), and then grab the image dimensions
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=600)
    #cv2_imshow(image)
    #print(image.shape)
    (h, w) = image.shape[:2]

    #imm=cv2.resize(image,(300,300))
    #cv2_imshow(image)
    #print(image.shape)

    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
      cv2.resize(image, (300, 300)), 1.0, (300, 300),
      (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    print(detections)
    print(detections.shape)
    print(len(detections))

    # ensure at least one face was found
    if len(detections) > 0:
      # we're making the assumption that each image has only ONE face, so find the bounding box with the largest probability
      i = np.argmax(detections[0, 0, :, 2])
      confidence = detections[0, 0, i, 2]

      # ensure that the detection with the largest probability also means our minimum probability test (thus helping filter out weak detections)
      if confidence > 0.5:
        # compute the (x, y)-coordinates of the bounding box for the face
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # extract the face ROI and grab the ROI dimensions
        face = image[startY:endY, startX:endX]
        (fH, fW) = face.shape[:2]

        # ensure the face width and height are sufficiently large
        if fW < 20 or fH < 20:
          continue

        # construct a blob for the face ROI, then pass the blob through our face embedding model to obtain the 128-d quantification of the face
        faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
          (96, 96), (0, 0, 0), swapRB=True, crop=False)
        embedder.setInput(faceBlob)
        vec = embedder.forward()

        # add the name of the person + corresponding face embedding to their respective lists
        knownNames.append(name)
        knownEmbeddings.append(vec.flatten())
        total += 1
      else:
        

  # dump the facial embeddings + names to disk
  print("[INFO] serializing {} encodings...".format(total))
  data = {"embeddings": knownEmbeddings, "names": knownNames}
  f = open("/content/drive/MyDrive/Myphotos/output/embeddings.pickle", "wb")
  f.write(pickle.dumps(data))
  f.close()

def trainSVM():
  print("starting training Process")
  data = pickle.loads(open("/content/drive/MyDrive/Myphotos/output/embeddings.pickle", "rb").read())
  le = LabelEncoder()
  labels = le.fit_transform(data["names"])

  print("[INFO] training model...")
  recognizer = SVC(C=1.0, kernel="linear", probability=True)
  #print(data['embeddings'])
  if data['embeddings']:
    recognizer.fit(data["embeddings"], labels)

  print("Svm trained")
  f = open("/content/drive/MyDrive/Myphotos/output/recognizer.pickle", "wb")
  f.write(pickle.dumps(recognizer))
  f.close()

  f = open("/content/drive/MyDrive/Myphotos/output/le.pickle", "wb")
  f.write(pickle.dumps(le))
  f.close()


In [ ]:
embeddingsGenrerator()
trainSVM()